In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import json
from scipy.spatial.transform import Rotation as R  # rotation axis ??? left-hand / clockwise

from base import Camera
import numpy as np

np.set_printoptions(suppress=True)

In [6]:
with open('./data/solo/sequence.0/step0.frame_data.json') as json_file:
    data = json.load(json_file)
cameras = data['captures']

for camera_info in cameras:
    print(camera_info['position'])

    camera = Camera(
        position=camera_info['position'],
        quaternion=camera_info['rotation'],
        resolution=camera_info['dimension'],
    )

[0.0, 2.0, 5.0]
[0.0, 2.0, 5.0]


In [42]:
def qm(quaternion1, quaternion0):
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    
    return np.array([
        -x1*x0 - y1*y0 - z1*z0 + w1*w0,
         x1*w0 + y1*z0 - z1*y0 + w1*x0,
        -x1*z0 + y1*w0 + z1*x0 + w1*y0,
         x1*y0 - y1*x0 + z1*w0 + w1*z0
    ])

q = np.array([0, 1, 0, 0])
abc = [
    np.array([0, 0, 0.2, 0]),
]
d = np.array([0, 2, 5])

_inv = np.array([1, -1, -1, -1])
print(
    qm(qm(q, abc[0]), q*_inv)
)

[ 0.   0.  -0.2  0. ]


In [34]:
r = R.from_euler('y', 135, degrees=True)
print(r.as_quat())
print(r.as_euler('xyz', degrees=True))
print(r.as_euler('xzy', degrees=True))
print(r.as_euler('yxz', degrees=True))
print(r.as_euler('yzx', degrees=True))
print(r.as_euler('zxy', degrees=True))
print(r.as_euler('zyx', degrees=True))

[0.         0.92387953 0.         0.38268343]
[180.  45. 180.]
[  0.   0. 135.]
[135.   0.   0.]
[135.   0.   0.]
[  0.   0. 135.]
[-180.   45.  180.]


In [22]:
r = R.from_euler('y', 90, degrees=True)
r.as_quat()

array([0.        , 0.70710678, 0.        , 0.70710678])